<span style="font-family: Arial; font-size: 14pt;"><b>Intermediate Training and Fine-tuning of BERT on Geneva Bible</b></span><br><br>
<span style="font-family: Arial; font-size: 12pt;">Author: Lucas Ma</span>

The first chunk of code loads the pretrained model in the repo. Note: the directory of the trained model is purposefully untracked by git. Please do not attempt to track it by "git add XXX" as it prevents you from pushing successfully to the remote main branch i.e. our GitHub repo.

In [1]:
import torch
from transformers import AutoModelForMaskedLM, AutoTokenizer

# Load the fine-tuned MacBERTh model and tokenizer
# model_name = './fine-tuned-MacBERTh'  # Path to your fine-tuned model
model_name = "emanjavacas/MacBERTh"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForMaskedLM.from_pretrained(model_name)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


Now get embedding.

In [2]:
def get_word_embedding(word, tokenizer, model):
    # Tokenize the input word
    inputs = tokenizer(word, return_tensors='pt')
    
    # Get the hidden states from the model
    with torch.no_grad():
        outputs = model(**inputs, output_hidden_states=True)
    
    # Extract the last hidden state for the input token
    hidden_states = outputs.hidden_states[-1]  # Shape: (batch_size, sequence_length, hidden_size)
    
    # Get the embedding for the first token (the input word)
    word_embedding = hidden_states[0, 1, :]  # Assuming the word is at position 1
    return word_embedding

# Example words
word1 = "Virginia"
word2 = "sathan"

# Get embeddings for the words
embedding1 = get_word_embedding(word1, tokenizer, model)
embedding2 = get_word_embedding(word2, tokenizer, model)


Now compute cosine similarity.

In [3]:
import torch.nn.functional as F

def cosine_similarity(embedding1, embedding2):
    # Ensure the embeddings are normalized
    embedding1 = F.normalize(embedding1, p=2, dim=0)
    embedding2 = F.normalize(embedding2, p=2, dim=0)
    
    # Compute cosine similarity
    cosine_sim = torch.dot(embedding1, embedding2).item()
    return cosine_sim

# Compute the cosine similarity between the embeddings
cosine_sim = cosine_similarity(embedding1, embedding2)
print(f"Cosine Similarity between '{word1}' and '{word2}': {cosine_sim}")


Cosine Similarity between 'Virginia' and 'sathan': 0.9525505900382996
